<a href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/autosegment_CT_Heart_OARs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the Heart sub-structures on a lung CT scan using pyCERR.

## Requirements
* Python>=3.8
* Applying this model requires access to a GPU.  

## AI model
* The segmentation model was trained and validated on CT scans used for RT planning. It does not work optimally on diagnostic CTs or scans in positions other than Head First Supine.
* The trained model is distributed along with python libraries and other dependencies via a conda package.

### Running the model

Update locations of input data and model directorues in section 2 of this notebook.
* Conda packge is location: condaEnvDir
* Inference script location: wrapperPath
* Inference script args
```python
!python {wrapperPath} {input_nii_directory} {output_nii_directory}
```

### License
By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.
    
This software is for research purposes only and has not been approved for clinical use.

Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.

You may publish papers and books using results produced using software provided that you reference the appropriate citations:
*  Heart sub-structures model: https://doi.org/10.1016/j.phro.2020.05.009
*  CERR library of model implementations: https://doi.org/10.1016/j.ejmp.2020.04.011
*  CERR software: https://doi.org/10.1118/1.1568978
*  CERR radiomics: https://doi.org/10.1002/mp.13046


YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.

All Technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain such licenses to export, re-export, or import as may be required after delivery to you.



##  Define paths for input DICOM, output segmentation and session directories

Specify paths to the DICOM input data, desired output directory, and temporary (session) directory used to store intermediate results. The input data is structures such that DICOM per patient scan is in an individual directory.

&nbsp;&nbsp;Input Directory  
&nbsp;&nbsp;&nbsp;&nbsp;Pat1  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;img1.dcm  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;img2.dcm  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;....  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;....  
&nbsp;&nbsp;&nbsp;&nbsp;Pat2  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;img1.dcm  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;img2.dcm  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;....  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;....  

In [1]:
import os
# work dir
workDir = r'/home/jupyter' # r'/content' for Colab # r'AI_workshop/heart_model' # local

# Location of dicom data
dataDownloadDir = os.path.join(workDir,'sampleData','testPlanningCTdicom')

os.makedirs(dataDownloadDir,exist_ok=True)

In [5]:
# Location of LungOAR conda environment
heartCondaEnvDir = os.path.join(workDir, 'pretrainedHeartModel')

# Path to conda environment activate script
heartEnvActivateScript = os.path.join(heartCondaEnvDir, 'bin', 'activate')

In [6]:
wrapperInstallDir = os.path.join(workDir, 'CT_cardiac_structures_deeplab')
wrapperPath = os.path.join(wrapperInstallDir, 'model_wrapper','runSegmentation.py')
modelWeightZipPath = os.path.join(wrapperInstallDir,'model.gz')
modelWeightPath = os.path.join(wrapperInstallDir,'model')
modelHash = 'H4sIAAAAAAAAAwXBUQqAIBAFwP8O46YlZLexLTVELN9S0umbSSIXVqKCzKy22hXXQki+HTtBvJxM1Zq5I1o4fY+hvdnrMTwIi5mcRYwqfsMPzAV1z0gAAAA='

# Downloads

## Download planning CT DICOM from ***dataUrl*** to ***dataDownloadDir***

In [ ]:
dataUrl='http://path.to/data'
! wget -O sampleData.gz -L {dataUrl}
! tar xf sampleData.gz -C {dataDownloadDir}
! rm sampleData.gz

In [4]:
#Paths to input data and conda env with pre-trained models
#inputDicomPath = os.path.join(dataDownloadDir,'rtog-0617','0617_test')  # Replace with apropriate path to your dataset

inputDicomPath = os.path.join(dataDownloadDir,'0617_test')
outputDicomPath = os.path.join(workDir, 'AIoutput')
sessionPath = os.path.join(workDir, 'temp')

if not os.path.exists(outputDicomPath):
  os.mkdir(outputDicomPath)

if not os.path.exists(sessionPath):
  os.mkdir(sessionPath)


## Download the pre-packaged Anaconda environment to *heartCondaEnvDir*

In [ ]:
# Download packaged environment for the Heart segmentation model
boxHash = 'H4sIAAAAAAAAAwXBUQqAIAwA0P8O4xAKotvoCjWzqVuZnr73vEjmDSBxRFSWPoWUgL2pxw4sRgLCfAXNNMyS8W6llxaf16WziV11lErKjekHlA19CEgAAAA='
saveFileName = 'ct_heart_oar.tar.gz'

!mkdir -p {heartCondaEnvDir}
!wget `base64 -d <<<{boxHash} | gunzip` -O {saveFileName}
!tar xf {saveFileName} -C {heartCondaEnvDir}
!rm {saveFileName}


## Download the inference script and model weights to *wrapperInstallDir*

In [ ]:
!git clone https://github.com/cerr/CT_cardiac_structures_deeplab.git {wrapperInstallDir}
!wget -O {modelWeightZipPath} -L `base64 -d <<<{modelHash} | gunzip`
!tar xf {modelWeightZipPath} -C {wrapperInstallDir}
!rm {modelWeightZipPath}

# Install pyCERR

pyCERR is used for pre and post-processing of DICOM as required by the model.

In [ ]:
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"

# Functions for data pre- and post-processing

## Crop scan to Lung extents

In [11]:
from cerr.dataclasses import structure as cerrStr
from cerr.contour import rasterseg as rs
from cerr.utils import aiPipeline, mask
import numpy as np

def processInputData(scanNum, planC, lungNameList=['Lung_total', 'Lung_L', 'Lung_R']):

    if isinstance(lungNameList, str):
        lungNameList = [lungNameList]

    # Extract scanArray
    scan3M = planC.scan[scanNum].getScanArray()
    mask3M = np.zeros(scan3M.shape, dtype=bool)

    # List of Structure names
    strNames = [s.structureName for s in planC.structure]
    numOrigStructs = len(strNames)

    # Get total lung mask
    for lungName in lungNameList:
        lungInd = cerrStr.getMatchingIndex(lungName, strNames, 'exact')
        if len(lungInd) > 0:
            # Get lung extents
            mask3M = mask3M | rs.getStrMask(lungInd[0], planC)

    if not np.any(mask3M):
        raise Exception('Lung contour name did not match any structures in planC')

    # Create cropped scan
    rmin,rmax,cmin,cmax,smin,smax,_ = mask.compute_boundingbox(mask3M)
    x,y,z = planC.scan[0].getScanXYZVals()
    xCropV = x[cmin:cmax]
    yCropV = y[rmin:rmax]
    zCropV = z[smin:smax]
    scan3M = planC.scan[0].getScanArray()
    scanCrop3M = scan3M[rmin:rmax,cmin:cmax,smin:smax]

    return scanCrop3M, (xCropV, yCropV, zCropV)


## Import AI segmentations to planC and retain only the largest connected component for each structure

In [18]:
#Import label map to CERR
import glob
from cerr import plan_container as pc

atriaLabelDict = {1: 'DL_Atria'}
heartSubSegDict = {2: 'DL_AORTA', 3: 'DL_LA',
                   4: 'DL_LV', 5: 'DL_RA',
                   6: 'DL_RV', 7: 'DL_IVC',
                   8: 'DL_SVC', 9: 'DL_PA'}
heartSegDict = {1: 'DL_heart'}
periLabelDict = {1: 'DL_Pericardium'}
ventriLabelDict = {1: 'DL_Ventricles'}

def postProcAndImportSeg(outputDir,procScanNum,scanNum,planC):
    niiGlob = glob.glob(os.path.join(outputDir,'*.nii.gz'))
    for segFile in niiGlob:
        print('Importing ' + segFile + '...')
        # Get segFile name
        if 'heart.nii.gz' in segFile:
            strToLabelMap = heartSubSegDict
        elif 'heartStructure.nii.gz' in segFile:
            strToLabelMap = heartSegDict
        elif 'atria.nii.gz' in segFile:
            strToLabelMap = atriaLabelDict
        elif 'pericardium.nii.gz' in segFile:
            strToLabelMap = periLabelDict
        elif 'ventricles.nii.gz' in segFile:
            strToLabelMap = ventriLabelDict
        numLabel = len(strToLabelMap)
        numStrOrig = len(planC.structure)
        planC = pc.load_nii_structure(segFile, scanNum, planC, \
                                  labels_dict = strToLabelMap)
        numStructs = len(planC.structure)
        cpyStrNumV = np.arange(numStrOrig,numStructs)
        numConnComponents = 1
        for structNum in cpyStrNumV:
            _, planC = cerrStr.getLargestConnComps(structNum, numConnComponents, planC, \
                                            saveFlag=True, replaceFlag=True)
            
    return planC

# Segment OARs for all the CT scans located at *inputDicomPath*

In [19]:
#%%capture
import os
import subprocess
import numpy as np
import cerr
from cerr import plan_container as pc
from cerr.dataclasses import scan as cerrScn
from cerr.utils.aiPipeline import createSessionDir
from cerr.dcm_export import rtstruct_iod

# Loop over pyCERR files
fileList = os.listdir(inputDicomPath)
numFiles = len(fileList)
modality = 'CT SCAN'
lungNameList = ['LUNG_TOTAL', 'LUNG_CNTR', 'LUNG_IPSI']

for iFile in range(numFiles):

    dcmDir = os.path.join(inputDicomPath,fileList[iFile])

    # Create session dir to store temporary data
    modInputPath, modOutputPath = createSessionDir(sessionPath, inputDicomPath)

    # Import DICOM to planC
    planC = pc.load_dcm_dir(dcmDir)

    # Identify scan index in  planC
    scanIdS = {"imageType": modality}
    matchScanV = aiPipeline.getScanNumFromIdentifier(scanIdS, planC, False)
    scanNum = matchScanV[0]

    # Pre-process data
    procScan3M, resizeGridS = processInputData(scanNum, planC, lungNameList)
    planC = pc.import_scan_array(procScan3M, resizeGridS[0], \
            resizeGridS[1], resizeGridS[2], modality, scanNum, planC)
    procScanNum = len(planC.scan) - 1

    # Export inputs to NIfTI
    scanFilename = os.path.join(modInputPath, f"{fileList[iFile]}_scan_3D.nii.gz")
    planC.scan[procScanNum].save_nii(scanFilename)

    numOrigStructs = len(planC.structure)

    # Apply model
    runScript = "source " + heartEnvActivateScript + " && python " + wrapperPath \
                  + " " + modInputPath + " " + modOutputPath
    print(runScript)
    subprocess.run(runScript,
                 capture_output=False,
                  shell=True,
                  executable="/bin/bash")

    # Import results to planC
    planC = postProcAndImportSeg(modOutputPath,procScanNum,scanNum,planC)

    numStructs = len(planC.structure)

    # Export segmentations to DICOM
    structFileName = fileList[iFile] + '_AI_seg.dcm'
    structFilePath = os.path.join(outputDicomPath,structFileName)
    structNumV = np.arange(numOrigStructs, numStructs)
    indOrigV = np.array([cerrScn.getScanNumFromUID(planC.structure[structNum].assocScanUID, planC) for structNum in structNumV], dtype=int)
    origIndsToExportV = structNumV[indOrigV == scanNum]
    seriesDescription = "AI Generated"
    exportOpts = {'seriesDescription': seriesDescription}
    rtstruct_iod.create(origIndsToExportV,structFilePath,planC,exportOpts)


('0617-292370', '0617-292370', '1.3.6.1.4.1.14519.5.2.1.6329.6468.180596963996965825122162035932', '1.3.6.1.4.1.14519.5.2.1.6329.6468.166829248218190705316144387032', 'RTDOSE', 'RTDOSE', 'RTDOSE', 'RTDOSE', 'RTDOSE', 'RTDOSE', 'RTDOSE')
('0617-292370', '0617-292370', '1.3.6.1.4.1.14519.5.2.1.6329.6468.180596963996965825122162035932', '1.3.6.1.4.1.14519.5.2.1.6329.6468.256371526114994213353892237901', 'RTPLAN', 'RTPLAN', 'RTPLAN', 'RTPLAN', 'RTPLAN', 'RTPLAN', 'RTPLAN')
('0617-292370', '0617-292370', '1.3.6.1.4.1.14519.5.2.1.6329.6468.180596963996965825122162035932', '1.3.6.1.4.1.14519.5.2.1.6329.6468.314188004903460989626820333570', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT', 'CT')


/cluster/home/aptea/miniconda3/envs/pycerr/lib/python3.11/site-packages/pydicom/valuerep.py:443: UserWarning: Invalid value for VR IS: '23.617'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/cluster/home/aptea/miniconda3/envs/pycerr/lib/python3.11/site-packages/pydicom/valuerep.py:1286: UserWarning: Value "23.617" is not valid for elements with a VR of IS
  warnings.warn(msg)


('0617-292370', '0617-292370', '1.3.6.1.4.1.14519.5.2.1.6329.6468.180596963996965825122162035932', '1.3.6.1.4.1.14519.5.2.1.6329.6468.880278915833505391469023528647', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT')
source AI_workshop/heart_model/pretrainedHeartModel/bin/activate && python AI_workshop/heart_model/CT_cardiac_structures_deeplab/model_wrapper/runSegmentation.py AI_workshop/heart_model/temp/session0617_test233545703.733/inputNii AI_workshop/heart_model/temp/session0617_test233545703.733/outputNii
Starting Inference:
atria
Found 74 images in scan volume
GPU DEVICE COUNT
1
Cuda Available?
True
Using GPU...
***Model Load Time***
3.7947988510131836
Starting inference...


:   3%|▎         | 2/74 [00:00<00:04, 16.61it/s]

heart
Found 74 images in scan volume
GPU DEVICE COUNT
1
Cuda Available?
True
Using GPU...
***Model Load Time***
0.43782567977905273
Starting inference...


...

pericardium
Found 83 images in scan volume
GPU DEVICE COUNT
1
Cuda Available?
True
Using GPU...
***Model Load Time***
0.45183348655700684
Starting inference...


:   2%|▏         | 2/83 [00:00<00:05, 16.08it/s]

ventricles
Found 83 images in scan volume
GPU DEVICE COUNT
1
Cuda Available?
True
Using GPU...
***Model Load Time***
0.4396359920501709
Starting inference...


: 100%|██████████| 83/83 [00:04<00:00, 17.84it/s]


***Total Time***
35.3272385597229
Importing AI_workshop/heart_model/temp/session0617_test233835853.439/outputNii/0617-305105_09-09-2000-66193_scan_3D_heart.nii.gz...
Importing AI_workshop/heart_model/temp/session0617_test233835853.439/outputNii/0617-305105_09-09-2000-66193_scan_3D_heartStructure.nii.gz...
Importing AI_workshop/heart_model/temp/session0617_test233835853.439/outputNii/0617-305105_09-09-2000-66193_scan_3D_pericardium.nii.gz...
Importing AI_workshop/heart_model/temp/session0617_test233835853.439/outputNii/0617-305105_09-09-2000-66193_scan_3D_atria.nii.gz...
Importing AI_workshop/heart_model/temp/session0617_test233835853.439/outputNii/0617-305105_09-09-2000-66193_scan_3D_ventricles.nii.gz...
Writing RTSTRUCT file ... AI_workshop/heart_model/AIoutput/0617-305105_09-09-2000-66193_AI_seg.dcm
File saved.


# Optional - Download the outputDicomPath to Workspace bucket

In [ ]:
# workspaceBucket = os.environ['WORKSPACE_BUCKET']
# !gcloud storage cp -r {outputDicomPath} {workspaceBucket}

# Display results for the last CT scan

## Display using matplotlib

In [20]:
from cerr.viewer import showMplNb

showMplNb(scanNum, origIndsToExportV, planC,\
          windowCenter=-400, windowWidth=2000)

interactive(children=(IntSlider(value=63, description='slcNum', max=189, min=-63), Text(value='axial', descrip…

interactive(children=(IntSlider(value=256, description='slcNum', max=768, min=-256), Text(value='sagittal', de…

interactive(children=(IntSlider(value=256, description='slcNum', max=768, min=-256), Text(value='coronal', des…